In [7]:
# 1.Daily covid cases

import warnings
warnings.simplefilter(action='ignore')

import pandas as pd
import numpy as np
import datetime
# df=pd.read_excel(r'DataSource/airports.xlsx')
# df=df.drop(columns=['Unit'])
df=pd.read_excel(r'DataSource/flights.xlsx')
for col in ['更新时间','来源','频率','单位']:
    if col in df.columns:
        df=df.drop(columns=col)
df=df.dropna(subset=['指标名称'])
c1=df['指标名称'].str.contains('数据来源')
df=df[~c1]
df['指标名称']=df['指标名称'].str.replace('机场进出港航班:','')
df[['城市', '机场']] = df['指标名称'].str.split("/", expand = True)
df=df.drop(columns=['指标名称'])
cols=list(df.columns[-2:])+sorted(list(df.columns[:-2]),reverse=False)
df=df[cols]

df.to_hdf('./Processed_Data/airports.h5',key='raw')
df

,城市,机场,2019-01-01,2019-01-02,2019-01-03,2019-01-04,2019-01-05,2019-01-06,2019-01-07,2019-01-08,...,2023-01-06,2023-01-07,2023-01-08,2023-01-09,2023-01-10,2023-01-11,2023-01-12,2023-01-13,2023-01-14,2023-01-15
0,三明,沙县,NaN,10.0,NaN,11.0,NaN,10.0,10.0,NaN,...,8.0,6.0,6.0,7.0,8.0,8.0,8.0,8.0,8.0,8.0
1,无锡,硕放,134.0,141.0,142.0,145.0,144.0,145.0,147.0,148.0,...,115.0,121.0,132.0,135.0,139.0,147.0,152.0,158.0,161.0,165.0
2,宜昌,三峡,45.0,52.0,53.0,56.0,56.0,56.0,57.0,58.0,...,44.0,47.0,50.0,52.0,56.0,60.0,62.0,66.0,68.0,72.0
3,广州,白云,1261.0,1286.0,1285.0,1292.0,1292.0,1296.0,1294.0,1298.0,...,649.0,676.0,732.0,757.0,800.0,842.0,887.0,935.0,981.0,1023.0
4,抚远,东极,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,义乌,None,35.0,37.0,35.0,36.0,37.0,37.0,36.0,38.0,...,31.0,32.0,36.0,38.0,39.0,41.0,42.0,44.0,46.0,46.0
229,呼和浩特,白塔,206.0,223.0,224.0,230.0,229.0,232.0,233.0,234.0,...,118.0,125.0,135.0,141.0,145.0,153.0,161.0,167.0,173.0,179.0
230,北京,大兴,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,436.0,454.0,487.0,497.0,521.0,549.0,578.0,605.0,631.0,653.0
231,鞍山,腾鳌,6.0,8.0,6.0,8.0,6.0,8.0,8.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
ap=pd.read_hdf('./Processed_Data/airports.h5',key='raw')
ap=ap.groupby(by='城市').sum().reset_index()
lastcol=ap.columns[-1]
ap=ap.sort_values(by=lastcol,ascending=False).head(50)
ap=pd.melt(ap,id_vars=['城市'],var_name='date',value_name='air_traffic')
ap=ap[ap['air_traffic']>0]

ap['date'] = pd.to_datetime(ap['date']).dt.date
ap=ap.sort_values(by=['城市','date'],ascending=True)
ap['date']=ap['date'].astype('str')

def getdt(d):
    return str(d)[-5:]
ap['dt']=ap['date'].apply(getdt)

ap['ap_last7d_mean']=ap.groupby('城市')['air_traffic'].transform(lambda x: x.rolling(7, 1).mean())
ap['ap_WoW']=ap['ap_last7d_mean']/ap.groupby('城市')['ap_last7d_mean'].transform(lambda x: x.shift(periods=7)) -1

c1=ap['date']>='2018-11-01'
c2=ap['date']<='2019-10-01'

ap_19=ap[c1&c2]
ap_19=ap_19[['城市', 'dt','ap_last7d_mean']]
ap_19['ap_last7d_mean']=ap.groupby('城市')['ap_last7d_mean'].transform(lambda x: x.shift(periods=-14)) #调整CNY
ap_19=ap_19.rename(columns={'ap_last7d_mean':'ap_19_rolling_mean'})

c2=ap['date']>'2022-05-01'
ap_22=ap[c2]
ap_22=ap_22.merge(ap_19,how='left',on=['城市','dt'])

ap_22['ap_status']=ap_22['ap_last7d_mean']/ap_22['ap_19_rolling_mean']

ap_19.to_clipboard()

In [3]:
# # TY 机场 vs. 2019
# %load_ext autoreload
# %autoreload 2
# from colored_list import dafeng_city,jidui_city,riqi
# riqi='2023-01-08'

# df_covid=pd.read_hdf('Processed_Data/Baidu_covid_progress.h5',key='raw')
# o=ap_22.merge(df_covid,on=['城市'],how='left')

# o=o[o['ap_status']<1.5][o['date']==riqi][['城市','progress_2','ap_status']] 

# o['size']=np.nan
# o['group']='其他'

# c1=o['城市'].isin(dafeng_city)
# o.loc[c1,'group']='本波疫情基本结束'

# o=o.sort_values(by=['group','GDP'],ascending=False)
# o.to_clipboard(index=False,header=None)



# print(riqi)
# print(o)

In [4]:
# # TY 机场数据 WoW
# # %load_ext autoreload
# # %autoreload 2
# # from colored_list import dafeng_city,jidui_city,riqi

# df_covid=pd.read_hdf('Processed_Data/Baidu_covid_progress.h5',key='raw')
# o=ap_22.merge(df_covid,on=['城市'],how='left')

# o=o[o['date']==riqi][['城市','progress_2','ap_WoW']] 

# o['size']=np.nan
# o['group']='其他'

# c1=o['城市'].isin(dafeng_city)
# o.loc[c1,'group']='本波疫情基本结束'

# o=o.sort_values(by='group',ascending=False)

# o.to_clipboard(index=False,header=None)
# print(riqi)

In [4]:
# TY 机场数据 WoW vs. GDP
%load_ext autoreload
%autoreload 2
from colored_list import dafeng_city,jidui_city,riqi
riqi='2023-01-15'
city_popu=pd.read_hdf('Processed_Data/citypopu.h5',key='raw2')
o=ap_22.merge(city_popu,on=['城市'],how='left')

o=o[o['date']==riqi][['城市','GDP','ap_WoW']] 

o['size']=np.nan
o['group']='其他'

c1=o['城市'].isin(dafeng_city)
o.loc[c1,'group']='本波疫情基本结束'

o=o.sort_values(by=['group','GDP'],ascending=False)

o.to_clipboard(index=False,header=None)
print(riqi)

2023-01-15


In [5]:
# TY 机场数据 as of 19 vs. GDP
# %load_ext autoreload
# %autoreload 2
# from colored_list import dafeng_city,jidui_city,riqi

city_popu=pd.read_hdf('Processed_Data/citypopu.h5',key='raw2')
o=ap_22.merge(city_popu,on=['城市'],how='left')

o=o[o['date']==riqi][o['ap_status']<1.5][['城市','GDP','ap_status']] 

o['size']=np.nan
o['group']='其他'

c1=o['城市'].isin(dafeng_city)
o.loc[c1,'group']='本波疫情基本结束'

o=o.sort_values(by=['group','GDP'],ascending=False)

d=ap_22[ap_22['date']==riqi][['城市','date','ap_status']].reset_index(drop=True).rename(columns={'ap_status':'airports'})
d['date']=pd.to_datetime(d['date'])
d.to_hdf('Processed_Data/city_compare.h5',key='airports')


o.to_clipboard(index=False,header=None)
print(riqi)

2023-01-15


In [7]:
# d1=pd.read_clipboard()
# d1.to_hdf('Processed_Data/tablehead.h5','airports')
d1=pd.read_hdf('Processed_Data/tablehead.h5','airports')

apo=pd.read_hdf('./Processed_Data/airports.h5',key='raw')
cols=[i for i in apo.columns if i >'2023-01-04']

d1.merge(apo[cols],on=['城市','机场'],how='left').drop(columns=['城市','机场']).to_clipboard(index=False)

In [68]:

citytier=pd.read_hdf('./Processed_Data/citypopu.h5',key='citytier')


bytier=ap_22.merge(citytier,on='城市',how='left')
bytier['ap_last7d_mean']=bytier['ap_last7d_mean']*7
bytier['ap_19_rolling_mean']=bytier['ap_19_rolling_mean']*7
bytier['CityTier']=bytier['CityTier'].str.replace('四线城市','三线城市')
bytier=bytier.groupby(['CityTier','date']).sum().reset_index()[['CityTier','date','ap_last7d_mean','ap_19_rolling_mean']]
bytier['status']=bytier['ap_last7d_mean']/bytier['ap_19_rolling_mean']
bytier['status']=bytier['status'].transform(lambda x: x.rolling(7, 1).mean())
bytier=pd.pivot_table(bytier,index='date',columns='CityTier',values='status',aggfunc='mean').reset_index()
bytier.sort_values(by='date').tail(20)

alldata=ap_22.merge(citytier,on='城市',how='left')
alldata['ap_last7d_mean']=alldata['ap_last7d_mean']*7
alldata['ap_19_rolling_mean']=alldata['ap_19_rolling_mean']*7
alldata=alldata[~alldata['CityTier'].isnull()].groupby(['date']).sum().reset_index()[['date','ap_last7d_mean','ap_19_rolling_mean']]
alldata['status']=alldata['ap_last7d_mean']/alldata['ap_19_rolling_mean']
alldata['status']=alldata['status'].transform(lambda x: x.rolling(7, 1).mean())
alldata=alldata[['date','status']].rename(columns={'status':'all'})

bytier.merge(alldata,how='left',on='date').tail(15).to_clipboard(index=False)



In [74]:
pd.pivot_table(ap_22.merge(citytier,on='城市',how='left'),index=['CityTier','城市'],columns='date',values='ap_last7d_mean',aggfunc='count').to_clipboard()